In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp

In [6]:
df=pd.read_csv("heart.csv")

In [7]:
df.isnull().mean()


Age               0.0
Sex               0.0
ChestPainType     0.0
RestingBP         0.0
Cholesterol       0.0
FastingBS         0.0
RestingECG        0.0
MaxHR             0.0
ExerciseAngina    0.0
Oldpeak           0.0
ST_Slope          0.0
HeartDisease      0.0
dtype: float64

In [8]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [9]:
columns_to_drop = ['FastingBS', 'RestingECG', 'ExerciseAngina']
df = df.drop(columns=columns_to_drop)
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,MaxHR,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,172,0.0,Up,0
1,49,F,NAP,160,180,156,1.0,Flat,1
2,37,M,ATA,130,283,98,0.0,Up,0
3,48,F,ASY,138,214,108,1.5,Flat,1
4,54,M,NAP,150,195,122,0.0,Up,0


In [11]:
print("Highest allowed",df['Cholesterol'].mean() + 3*df['Cholesterol'].std())
print("Lowest allowed",df['Cholesterol'].mean() - 3*df['Cholesterol'].std())

Highest allowed 526.9519979267629
Lowest allowed -129.35286938645797


In [13]:
df = df[(df['Cholesterol'] < 526.9519979267629) & (df['Cholesterol'] > -129.35286938645797)]
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,MaxHR,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,172,0.0,Up,0
1,49,F,NAP,160,180,156,1.0,Flat,1
2,37,M,ATA,130,283,98,0.0,Up,0
3,48,F,ASY,138,214,108,1.5,Flat,1
4,54,M,NAP,150,195,122,0.0,Up,0


In [45]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(df.iloc[:,0:8],df.iloc[:,-1],test_size=0.2,random_state=2)

In [46]:
xtrain

,Age,Sex,ChestPainType,RestingBP,Cholesterol,MaxHR,Oldpeak,ST_Slope
478,57,M,ASY,130,311,148,Yes,Flat
331,56,M,ASY,115,0,82,No,Up
187,41,M,ASY,120,237,138,Yes,Flat
318,61,M,ASY,130,0,77,Yes,Flat
815,68,M,NAP,118,277,151,Yes,Up
...,...,...,...,...,...,...,...,...
574,69,M,ASY,142,210,112,Yes,Flat
629,57,F,ASY,128,303,159,No,Up
524,55,M,ASY,150,160,150,No,Up
565,61,M,ASY,141,292,115,Yes,Flat


In [19]:
def convert(value):
    if value >= 1:
        return "Yes"
    else:
        return "No"

df['Oldpeak'] = df['Oldpeak'].apply(convert)

In [44]:
df = df[df["ST_Slope"] != "Down"]


In [58]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
xtrainsc = ohe.fit_transform(xtrain[['Sex','ChestPainType']])

# also the test data
xtestsc = ohe.fit_transform(xtest[['Sex','ChestPainType']])


c:\Users\Arunava Chakraborty\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Arunava Chakraborty\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [48]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(categories=[['Up','Flat']])
xtrainst = oe.fit_transform(xtrain[['ST_Slope']])

# also the test data
xtestst = oe.fit_transform(xtest[['ST_Slope']])

oe = OrdinalEncoder(categories=[['No','Yes']])
xtrainop = oe.fit_transform(xtrain[['Oldpeak']])

# also the test data
xtestop = oe.fit_transform(xtest[['Oldpeak']])

In [54]:
xtrainrem = xtrain.drop(columns=['Sex','ChestPainType','Oldpeak','ST_Slope'])
xtestrem = xtest.drop(columns=['Sex','ChestPainType','Oldpeak','ST_Slope'])


In [65]:
scaler = StandardScaler()
scaler.fit(xtrain[["MaxHR"]])
xtrain["MaxHR_scaled"] = scaler.transform(xtrain[["MaxHR"]])
xtest["MaxHR_scaled"] = scaler.transform(xtest[["MaxHR"]])
xtrain = xtrain.drop(columns=["MaxHR"])
xtest = xtest.drop(columns=["MaxHR"])


In [70]:
scaler = StandardScaler()
scaler.fit(xtrain[["RestingBP"]])
xtrain["RestingBP_scaled"] = scaler.transform(xtrain[["RestingBP"]])
xtest["RestingBP_scaled"] = scaler.transform(xtest[["RestingBP"]])
xtrain = xtrain.drop(columns=["RestingBP"])
xtest = xtest.drop(columns=["RestingBP"])

In [71]:
xtrain

,Age,Sex,ChestPainType,Cholesterol,Oldpeak,ST_Slope,MaxHR_scaled,RestingBP_scaled
478,57,M,ASY,311,Yes,Flat,0.438602,-0.157929
331,56,M,ASY,0,No,Up,-2.157659,-0.996416
187,41,M,ASY,237,Yes,Flat,0.045229,-0.716920
318,61,M,ASY,0,Yes,Flat,-2.354345,-0.157929
815,68,M,NAP,277,Yes,Up,0.556614,-0.828719
...,...,...,...,...,...,...,...,...
574,69,M,ASY,210,Yes,Flat,-0.977540,0.512860
629,57,F,ASY,303,No,Up,0.871312,-0.269728
524,55,M,ASY,160,No,Up,0.517277,0.960053
565,61,M,ASY,292,Yes,Flat,-0.859528,0.456961


In [72]:
xtraintransformed = np.concatenate((xtrainrem,xtrainsc,xtrainst,xtrainop),axis=1)
xtesttransformed = np.concatenate((xtestrem,xtestsc,xtestst,xtestop),axis=1)

In [73]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(xtraintransformed,ytrain)
predicted=lr.predict(xtesttransformed)



c:\Users\Arunava Chakraborty\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,predicted)

0.8187134502923976

In [ ]:
predicted


In [ ]:
ytest